In [33]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np
import ast
import time
import metrics

In [34]:
pd.set_option('display.max_colwidth', -1)

In [3]:
user_tokens = pd.read_csv('s3://recsys-challenge-2020/user_tokens.csv')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [ ]:
column_of_interest = ["engaging_user_id", "text_ tokens"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val_set_reply.csv', encoding="utf-8",
                     usecols= [1, 4])

In [282]:
column_of_interest = ["engaging_user_id", "text_ tokens"]
val_set_subset = pd.read_csv('s3://recsys-challenge-2020/val_set_reply.csv', encoding="utf-8",
                     usecols= [1, 4], nrows = 10000)

In [36]:
user_tokens.head()

,engaging_user_id,text_ tokens
0,00000B85AAF7DE172876FD96718C4469,['101\t12882\t10105\t24699\t10124\t12820\t169\t14624\t10862\t136\t102204\t10874\t102']
1,00000E0C9B364891CDE89ECFC54771DE,['101\t56898\t137\t60818\t20712\t10133\t37115\t32792\t19235\t15594\t131\t6054\t8403\t5115\t25317\t2271\t1913\t4348\t100\t119\t1921\t111786\t10055\t2072\t4796\t2072\t4796\t119\t1970\t111795\t18825\t14244\t4796\t23400\t7069\t1919\t22526\t1901\t100\t119\t119\t108\t52734\t38025\t35976\t56082\t10237\t11127\t14703\t10858\t10245\t11274\t11733\t11733\t119\t108\t12222\t11517\t23178\t14703\t20591\t11107\t14120\t131\t120\t120\t188\t119\t11170\t120\t173\t12022\t10575\t10477\t12022\t10305\t12396\t10112\t83026\t102']
2,0000142A2D3FCA2A1EAE0569CBEA8A11,['101\t56898\t137\t44798\t10858\t16896\t11083\t131\t94592\t12931\t14987\t158\t11273\t12931\t14120\t131\t120\t120\t188\t119\t11170\t120\t175\t10929\t11565\t11703\t11166\t10240\t10418\t13966\t27528\t102']
3,0000170273D2530A0DF580401CC32AE0,['101\t13999\t12845\t34396\t11088\t196\t10838\t43209\t10969\t20304\t74974\t118\t14120\t131\t120\t120\t188\t119\t11170\t120\t122\t72679\t11369\t37079\t12961\t16317\t108\t11566\t10253\t10136\t10575\t10350\t102']
4,00003C08A3D1CEF5F68EA6F17234F972,['101\t11589\t131\t146\t11337\t12253\t124\t19174\t10189\t50884\t50655\t112\t157\t62450\t10345\t96182\t15612\t10107\t10531\t10924\t119\t57757\t12253\t11152\t18922\t10901\t119\t119\t44718\t94722\t23040\t43039\t131\t100\t119\t10357\t10686\t37538\t10112\t10338\t36121\t131\t100\t119\t11454\t10108\t79400\t10107\t18121\t10162\t131\t100\t102']


In [37]:
val_set_subset.head()

,text_ tokens,engaging_user_id
0,101\t17713\t9405\t119152\t119002\t118823\t9531\t10530\t13767\t9665\t118649\t33323\t23466\t9405\t18623\t27023\t58303\t48345\t119\t119\t119\t9665\t118649\t33323\t23466\t11018\t9405\t119152\t9531\t11489\t9580\t118762\t9519\t119285\t130\t14040\t17655\t9580\t31531\t125\t14040\t18382\t68943\t9576\t16605\t58303\t48345\t119\t119\t119\t137\t49866\t27572\t10112\t11259\t12674\t11403\t108\t80426\t11403\t108\t9560\t22333\t14120\t131\t120\t120\t188\t119\t11170\t120\t171\t10731\t48205\t11259\t12396\t11211\t11281\t10174\t11090\t102,A4B1828B571271073C68DA93D1465484
1,101\t64002\t9247\t23665\t9251\t17342\t23990\t9764\t36553\t100543\t9358\t56789\t9247\t18108\t12310\t9485\t38709\t12178\t28911\t1729\t1729\t1729\t14120\t131\t120\t120\t188\t119\t11170\t120\t191\t19282\t10305\t83148\t13966\t10305\t13966\t11166\t10161\t102,F3A4C8EB81ADF3685E8F693037808A2C
2,101\t56898\t137\t44592\t11090\t13503\t10157\t131\t17144\t112\t187\t12820\t43662\t10271\t10950\t119\t119\t119\t14156\t10114\t13028\t10479\t10124\t10105\t14361\t10139\t10165\t16297\t96509\t13328\t11762\t10551\t136\t119\t108\t10882\t11759\t11273\t10280\t10305\t108\t31091\t34567\t10237\t10731\t10237\t100\t102,8BD011AB59AC04491687749245009C60
3,101\t137\t15595\t21840\t14590\t75268\t57493\t10759\t872\t13874\t32629\t28462\t102,F7BFBA56274FCBBBB76BDD36D0D6495B
4,101\t56898\t137\t13069\t93870\t10284\t11274\t12396\t168\t60801\t11211\t131\t9331\t30842\t10739\t9580\t118762\t9491\t66540\t17594\t100\t119\t9076\t68833\t8843\t19105\t18108\t9425\t12424\t9390\t119086\t9609\t11018\t8863\t100\t119\t9150\t12965\t8996\t26737\t101202\t9137\t11018\t9144\t9551\t23811\t16439\t100\t14120\t131\t120\t120\t188\t119\t11170\t120\t147\t11373\t11259\t10759\t12674\t10157\t11733\t10116\t11259\t10575\t102,10F6E035959B9AC156CED11E0FAED566


In [43]:
user_tokens_val_set_subset.head()

,text__tokens_x,engaging_user_id,text__tokens_y
0,101\t17713\t9405\t119152\t119002\t118823\t9531\t10530\t13767\t9665\t118649\t33323\t23466\t9405\t18623\t27023\t58303\t48345\t119\t119\t119\t9665\t118649\t33323\t23466\t11018\t9405\t119152\t9531\t11489\t9580\t118762\t9519\t119285\t130\t14040\t17655\t9580\t31531\t125\t14040\t18382\t68943\t9576\t16605\t58303\t48345\t119\t119\t119\t137\t49866\t27572\t10112\t11259\t12674\t11403\t108\t80426\t11403\t108\t9560\t22333\t14120\t131\t120\t120\t188\t119\t11170\t120\t171\t10731\t48205\t11259\t12396\t11211\t11281\t10174\t11090\t102,A4B1828B571271073C68DA93D1465484,NaN
1,101\t64002\t9247\t23665\t9251\t17342\t23990\t9764\t36553\t100543\t9358\t56789\t9247\t18108\t12310\t9485\t38709\t12178\t28911\t1729\t1729\t1729\t14120\t131\t120\t120\t188\t119\t11170\t120\t191\t19282\t10305\t83148\t13966\t10305\t13966\t11166\t10161\t102,F3A4C8EB81ADF3685E8F693037808A2C,NaN
2,101\t56898\t137\t44592\t11090\t13503\t10157\t131\t17144\t112\t187\t12820\t43662\t10271\t10950\t119\t119\t119\t14156\t10114\t13028\t10479\t10124\t10105\t14361\t10139\t10165\t16297\t96509\t13328\t11762\t10551\t136\t119\t108\t10882\t11759\t11273\t10280\t10305\t108\t31091\t34567\t10237\t10731\t10237\t100\t102,8BD011AB59AC04491687749245009C60,NaN
3,101\t137\t15595\t21840\t14590\t75268\t57493\t10759\t872\t13874\t32629\t28462\t102,F7BFBA56274FCBBBB76BDD36D0D6495B,['101\t15813\t885\t18321\t60270\t14070\t106\t119\t873\t15778\t35247\t865\t17110\t15778\t28462\t21042\t110334\t22022\t10569\t40265\t28546\t52331\t119\t10977\t898\t16940\t865\t17110\t15778\t28462\t21042\t119\t11186\t24578\t28546\t898\t66306\t11208\t22022\t13286\t16791\t117\t117\t117\t920\t119\t100\t119\t100\t28757\t100\t119\t100\t102']
4,101\t56898\t137\t13069\t93870\t10284\t11274\t12396\t168\t60801\t11211\t131\t9331\t30842\t10739\t9580\t118762\t9491\t66540\t17594\t100\t119\t9076\t68833\t8843\t19105\t18108\t9425\t12424\t9390\t119086\t9609\t11018\t8863\t100\t119\t9150\t12965\t8996\t26737\t101202\t9137\t11018\t9144\t9551\t23811\t16439\t100\t14120\t131\t120\t120\t188\t119\t11170\t120\t147\t11373\t11259\t10759\t12674\t10157\t11733\t10116\t11259\t10575\t102,10F6E035959B9AC156CED11E0FAED566,NaN


In [ ]:
user_tokens_val_set = \
pd.merge(val_set, user_tokens, how = 'left', left_on = 'engaging_user_id', right_on = 'engaging_user_id', sort=False)

In [ ]:
user_tokens_val_set.columns = [c.replace(' ', '_') for c in user_tokens_val_set.columns]

In [283]:
user_tokens_val_set_subset = \
pd.merge(val_set_subset, user_tokens, how = 'left', left_on = 'engaging_user_id', right_on = 'engaging_user_id', sort=False)

In [284]:
len(user_tokens_val_set_subset)

10000

In [285]:
user_tokens_val_set_subset.columns = [c.replace(' ', '_') for c in user_tokens_val_set_subset.columns]

In [286]:
user_tokens_val_set_subset.text__tokens_y.isnull().sum()

8702

In [287]:
mask = user_tokens_val_set_subset.text__tokens_y.notnull()
warm_ones = user_tokens_val_set_subset[mask]
warm_index = warm_ones.index
warm_index

Int64Index([   3,    6,    7,   11,   15,   27,   35,   51,   54,   60,
            ...
            9913, 9914, 9920, 9941, 9942, 9944, 9952, 9966, 9974, 9976],
           dtype='int64', length=1298)

In [288]:
user_tokens_val_set_subset.columns

Index(['text__tokens_x', 'engaging_user_id', 'text__tokens_y'], dtype='object')

In [289]:
len(user_tokens_val_set_subset)

10000

In [61]:
def calculate_average_old(row1, row2, index):
    if pd.isna(row1):
        return 0.028
    sum_tensors = torch.zeros([768], dtype=torch.float32)
    # row1 can be nan, as, there are cold users in validation set.
    tweet_token_list = ast.literal_eval(row1)
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        print(token_list_embeddings)
        sum_tensors = sum_tensors + token_list_embeddings
    avg = sum_tensors/len(row1)
    tweet_average_embedding = model(torch.tensor(list(map(int, row2.split('\t')))).unsqueeze(0))[0][0][0]
    score = torch.dot(avg, tweet_average_embedding)
    print(score)
    return score.detach().numpy().item(0)

In [290]:
def calculate_average(row1, row2, index):
    if index % 100 == 0:
        print(index)
    if pd.isna(row1):
        return 0.028
    sum_tensors = torch.zeros([768], dtype=torch.float32)
    tweet_token_list = ast.literal_eval(row1)
    for token_list in tweet_token_list:
        list_of_tokens = list(map(int, token_list.split('\t')))
        if len(list_of_tokens) > 512:
            pre_list = list_of_tokens[:511]
            post_list = list_of_tokens[511-len(list_of_tokens):]
            post_list.insert(0, 101)
            pre_token_list_embeddings = model(torch.tensor(pre_list).unsqueeze(0))[0][0][0]
            post_token_list_embeddings = model(torch.tensor(post_list).unsqueeze(0))[0][0][0]
            token_list_embeddings = (pre_token_list_embeddings + post_token_list_embeddings)/2.0
        else:
            token_list_embeddings = model(torch.tensor(list_of_tokens).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    avg = sum_tensors/len(row1)
    tweet_average_embedding = model(torch.tensor(list(map(int, row2.split('\t')))).unsqueeze(0))[0][0][0]
    score = torch.dot(avg, tweet_average_embedding)
    score = score/torch.norm(avg)
    likelihood = score/torch.norm(tweet_average_embedding)
    
#     likelihood*prior/(likelihood*prior) + (1-likelihood*1-prior)
    
    num = likelihood * 0.028
    
    unlikelihood = 1.0 - likelihood
    anti_score = unlikelihood * 0.972
    
    
    normalizing_factor = num + anti_score
    
    posterior = num/normalizing_factor
    
    print(posterior)
    return posterior.detach().numpy().item(0)

In [72]:
a = torch.tensor([3.0, 4.0])
torch.norm(a)

tensor(5.)

In [292]:
user_val_set_reply_score = pd.DataFrame()

In [ ]:
time1 = time.time()
user_val_set_reply_score['reply_score'] = user_tokens_val_set.apply \
(lambda z: calculate_average(z.text__tokens_y, z.text__tokens_x, z.name), axis = 1)
time2 = time.time()
print(time2 - time1)

In [293]:
time1 = time.time()
user_val_set_reply_score['reply_score'] = user_tokens_val_set_subset.apply \
(lambda z: calculate_average(z.text__tokens_y, z.text__tokens_x, z.name), axis = 1)
time2 = time.time()
print(time2 - time1)

0
tensor(0.0899, grad_fn=<DivBackward0>)
tensor(0.3556, grad_fn=<DivBackward0>)
tensor(0.1594, grad_fn=<DivBackward0>)
tensor(0.3302, grad_fn=<DivBackward0>)
tensor(0.2972, grad_fn=<DivBackward0>)
tensor(0.1096, grad_fn=<DivBackward0>)
tensor(0.0453, grad_fn=<DivBackward0>)
tensor(0.1296, grad_fn=<DivBackward0>)
tensor(0.1348, grad_fn=<DivBackward0>)
tensor(0.5500, grad_fn=<DivBackward0>)
tensor(0.1769, grad_fn=<DivBackward0>)
tensor(0.2086, grad_fn=<DivBackward0>)
tensor(0.1416, grad_fn=<DivBackward0>)
tensor(0.2191, grad_fn=<DivBackward0>)
tensor(0.1894, grad_fn=<DivBackward0>)
tensor(0.3188, grad_fn=<DivBackward0>)
100
tensor(0.0925, grad_fn=<DivBackward0>)
tensor(0.0369, grad_fn=<DivBackward0>)
tensor(0.1118, grad_fn=<DivBackward0>)
tensor(0.0504, grad_fn=<DivBackward0>)
tensor(0.0818, grad_fn=<DivBackward0>)
tensor(0.0616, grad_fn=<DivBackward0>)
tensor(0.1584, grad_fn=<DivBackward0>)
tensor(0.1071, grad_fn=<DivBackward0>)
tensor(0.1133, grad_fn=<DivBackward0>)
tensor(0.0976, grad

tensor(0.3593, grad_fn=<DivBackward0>)
tensor(0.0901, grad_fn=<DivBackward0>)
tensor(0.1644, grad_fn=<DivBackward0>)
tensor(0.3360, grad_fn=<DivBackward0>)
tensor(0.1721, grad_fn=<DivBackward0>)
1800
tensor(0.1044, grad_fn=<DivBackward0>)
tensor(0.0929, grad_fn=<DivBackward0>)
tensor(0.2654, grad_fn=<DivBackward0>)
tensor(0.1259, grad_fn=<DivBackward0>)
tensor(0.1790, grad_fn=<DivBackward0>)
tensor(0.2742, grad_fn=<DivBackward0>)
tensor(0.1907, grad_fn=<DivBackward0>)
tensor(0.1694, grad_fn=<DivBackward0>)
tensor(0.4971, grad_fn=<DivBackward0>)
tensor(0.1886, grad_fn=<DivBackward0>)
tensor(0.1270, grad_fn=<DivBackward0>)
tensor(0.0655, grad_fn=<DivBackward0>)
tensor(0.0265, grad_fn=<DivBackward0>)
tensor(0.2445, grad_fn=<DivBackward0>)
tensor(0.3379, grad_fn=<DivBackward0>)
tensor(0.0222, grad_fn=<DivBackward0>)
tensor(0.0737, grad_fn=<DivBackward0>)
tensor(0.2163, grad_fn=<DivBackward0>)
tensor(0.2097, grad_fn=<DivBackward0>)
tensor(0.3014, grad_fn=<DivBackward0>)
1900
tensor(0.2065, 

tensor(0.2889, grad_fn=<DivBackward0>)
tensor(0.2100, grad_fn=<DivBackward0>)
tensor(0.2588, grad_fn=<DivBackward0>)
3200
tensor(0.1648, grad_fn=<DivBackward0>)
tensor(0.0639, grad_fn=<DivBackward0>)
tensor(0.3567, grad_fn=<DivBackward0>)
tensor(0.2740, grad_fn=<DivBackward0>)
tensor(0.1258, grad_fn=<DivBackward0>)
tensor(0.0784, grad_fn=<DivBackward0>)
tensor(0.3382, grad_fn=<DivBackward0>)
tensor(0.1748, grad_fn=<DivBackward0>)
tensor(0.1518, grad_fn=<DivBackward0>)
tensor(0.1368, grad_fn=<DivBackward0>)
tensor(0.0657, grad_fn=<DivBackward0>)
tensor(0.0659, grad_fn=<DivBackward0>)
tensor(0.2089, grad_fn=<DivBackward0>)
tensor(0.1709, grad_fn=<DivBackward0>)
tensor(0.1198, grad_fn=<DivBackward0>)
tensor(0.2779, grad_fn=<DivBackward0>)
tensor(0.1471, grad_fn=<DivBackward0>)
3300
tensor(0.0410, grad_fn=<DivBackward0>)
tensor(0.3018, grad_fn=<DivBackward0>)
tensor(0.0513, grad_fn=<DivBackward0>)
tensor(0.1109, grad_fn=<DivBackward0>)
tensor(0.4404, grad_fn=<DivBackward0>)
tensor(0.1411, 

tensor(0.0840, grad_fn=<DivBackward0>)
4900
tensor(0.1325, grad_fn=<DivBackward0>)
tensor(0.1053, grad_fn=<DivBackward0>)
tensor(0.0674, grad_fn=<DivBackward0>)
tensor(0.0948, grad_fn=<DivBackward0>)
tensor(0.0756, grad_fn=<DivBackward0>)
tensor(0.0903, grad_fn=<DivBackward0>)
tensor(0.2911, grad_fn=<DivBackward0>)
tensor(0.1234, grad_fn=<DivBackward0>)
tensor(0.0947, grad_fn=<DivBackward0>)
tensor(0.1078, grad_fn=<DivBackward0>)
tensor(0.1079, grad_fn=<DivBackward0>)
tensor(0.0942, grad_fn=<DivBackward0>)
tensor(0.1351, grad_fn=<DivBackward0>)
tensor(0.1493, grad_fn=<DivBackward0>)
tensor(0.2039, grad_fn=<DivBackward0>)
tensor(0.1480, grad_fn=<DivBackward0>)
tensor(0.1239, grad_fn=<DivBackward0>)
tensor(0.1241, grad_fn=<DivBackward0>)
tensor(0.1020, grad_fn=<DivBackward0>)
tensor(0.1458, grad_fn=<DivBackward0>)
tensor(0.1758, grad_fn=<DivBackward0>)
tensor(0.2783, grad_fn=<DivBackward0>)
5000
tensor(0.0716, grad_fn=<DivBackward0>)
tensor(0.1806, grad_fn=<DivBackward0>)
tensor(0.1096, 

tensor(0.3443, grad_fn=<DivBackward0>)
tensor(0.0986, grad_fn=<DivBackward0>)
tensor(0.1207, grad_fn=<DivBackward0>)
tensor(0.0478, grad_fn=<DivBackward0>)
tensor(0.1157, grad_fn=<DivBackward0>)
tensor(0.3416, grad_fn=<DivBackward0>)
tensor(0.3042, grad_fn=<DivBackward0>)
tensor(0.2542, grad_fn=<DivBackward0>)
tensor(0.1738, grad_fn=<DivBackward0>)
tensor(0.1396, grad_fn=<DivBackward0>)
tensor(0.2420, grad_fn=<DivBackward0>)
6400
tensor(0.1383, grad_fn=<DivBackward0>)
tensor(0.3706, grad_fn=<DivBackward0>)
tensor(0.3632, grad_fn=<DivBackward0>)
tensor(0.1583, grad_fn=<DivBackward0>)
tensor(0.1102, grad_fn=<DivBackward0>)
tensor(0.0574, grad_fn=<DivBackward0>)
tensor(0.2016, grad_fn=<DivBackward0>)
tensor(0.1156, grad_fn=<DivBackward0>)
tensor(0.3911, grad_fn=<DivBackward0>)
tensor(0.0792, grad_fn=<DivBackward0>)
tensor(0.1002, grad_fn=<DivBackward0>)
tensor(0.0913, grad_fn=<DivBackward0>)
tensor(0.1948, grad_fn=<DivBackward0>)
tensor(0.1575, grad_fn=<DivBackward0>)
tensor(0.4923, grad_

tensor(0.2147, grad_fn=<DivBackward0>)
tensor(0.0977, grad_fn=<DivBackward0>)
tensor(0.2791, grad_fn=<DivBackward0>)
tensor(0.0875, grad_fn=<DivBackward0>)
tensor(0.0364, grad_fn=<DivBackward0>)
tensor(0.1321, grad_fn=<DivBackward0>)
tensor(0.1488, grad_fn=<DivBackward0>)
tensor(0.1087, grad_fn=<DivBackward0>)
tensor(-0.0019, grad_fn=<DivBackward0>)
tensor(0.1113, grad_fn=<DivBackward0>)
8000
tensor(0.4073, grad_fn=<DivBackward0>)
tensor(0.0440, grad_fn=<DivBackward0>)
tensor(-0.0016, grad_fn=<DivBackward0>)
tensor(0.1194, grad_fn=<DivBackward0>)
tensor(0.0385, grad_fn=<DivBackward0>)
tensor(0.0994, grad_fn=<DivBackward0>)
tensor(0.2988, grad_fn=<DivBackward0>)
tensor(0.0635, grad_fn=<DivBackward0>)
tensor(0.1030, grad_fn=<DivBackward0>)
tensor(0.1647, grad_fn=<DivBackward0>)
tensor(0.1164, grad_fn=<DivBackward0>)
8100
tensor(0.2679, grad_fn=<DivBackward0>)
tensor(0.0493, grad_fn=<DivBackward0>)
tensor(0.1368, grad_fn=<DivBackward0>)
tensor(0.0662, grad_fn=<DivBackward0>)
tensor(0.1462

tensor(0.1690, grad_fn=<DivBackward0>)
tensor(0.3206, grad_fn=<DivBackward0>)
tensor(0.1553, grad_fn=<DivBackward0>)
tensor(0.1126, grad_fn=<DivBackward0>)
tensor(0.1886, grad_fn=<DivBackward0>)
tensor(0.1187, grad_fn=<DivBackward0>)
tensor(0.0669, grad_fn=<DivBackward0>)
tensor(0.1318, grad_fn=<DivBackward0>)
tensor(0.0522, grad_fn=<DivBackward0>)
tensor(0.0436, grad_fn=<DivBackward0>)
tensor(0.0969, grad_fn=<DivBackward0>)
tensor(0.1847, grad_fn=<DivBackward0>)
tensor(0.3540, grad_fn=<DivBackward0>)
tensor(0.2442, grad_fn=<DivBackward0>)
tensor(0.1651, grad_fn=<DivBackward0>)
9800
tensor(0.1980, grad_fn=<DivBackward0>)
tensor(0.1233, grad_fn=<DivBackward0>)
tensor(0.0952, grad_fn=<DivBackward0>)
tensor(0.0869, grad_fn=<DivBackward0>)
tensor(0.1209, grad_fn=<DivBackward0>)
tensor(0.1052, grad_fn=<DivBackward0>)
tensor(0.1974, grad_fn=<DivBackward0>)
tensor(0.1659, grad_fn=<DivBackward0>)
tensor(0.0572, grad_fn=<DivBackward0>)
tensor(0.2095, grad_fn=<DivBackward0>)
tensor(0.1670, grad_

In [ ]:
user_val_set_reply_score['reply_score'] = user_tokens_val_set.apply \
(lambda z: calculate_average_old(z.text__tokens_y, z.text__tokens_x), axis = 1)

In [ ]:
user_val_set_reply_score.to_csv('s3://recsyschallenge2020/user_val_set_reply_score', index = False)

In [52]:
user_val_set_reply_score[user_val_set_reply_score.reply_score == 0.028]

,reply_score
0,0.028
1,0.028
2,0.028
4,0.028
5,0.028
...,...
995,0.028
996,0.028
997,0.028
998,0.028


### Compute Metrics

In [294]:
column_of_interest = ["reply_bool"]
val_set_subset = pd.read_csv('s3://recsys-challenge-2020/val_set_reply.csv', encoding="utf-8",
                     usecols= [6], nrows = 10000)

In [97]:
val_set_subset.loc[warm_index]

,reply_bool
3,0.0
6,0.0
7,0.0
11,1.0
15,0.0
...,...
952,0.0
970,0.0
980,1.0
988,0.0


In [ ]:
mask = user_val_set_reply_score.loc[val_set_subset[val_set_subset['reply_bool'] == 1.0].index].reply_score == 0.028
mask.value_counts()

In [85]:
warm_scores = user_val_set_reply_score[user_val_set_reply_score['reply_score'] != 0.028]

In [228]:
mask = user_val_set_reply_score.loc[val_set_subset.loc[warm_index][val_set_subset['reply_bool'] == 0.0].index].reply_score >= 0.028

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [230]:
mask = user_val_set_reply_score.loc[val_set_subset.loc[warm_index][val_set_subset['reply_bool'] == 0.0].index].reply_score

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [231]:
mask.mean()

0.15267635766898272

In [229]:
mask.value_counts()

True     97
False    6 
Name: reply_score, dtype: int64

In [232]:
mask = user_val_set_reply_score.loc[val_set_subset.loc[warm_index][val_set_subset['reply_bool'] == 1.0].index].reply_score >= 0.028

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [234]:
mask = user_val_set_reply_score.loc[val_set_subset.loc[warm_index][val_set_subset['reply_bool'] == 1.0].index].reply_score

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [233]:
mask.value_counts()

True    11
Name: reply_score, dtype: int64

In [235]:
mask.mean()

0.19554551724683156

In [295]:
pr = list(user_val_set_reply_score['reply_score'])

In [296]:
gt = list(val_set_subset['reply_bool'])

In [297]:
len(gt)

10000

In [298]:
len(pr)

10000

In [302]:
pr = [0.028]*10000

In [299]:
metrics.compute_prauc(pr,gt)

0.06277118766600823

In [303]:
metrics.compute_prauc(pr,gt)

0.5129

In [300]:
metrics.compute_rce(pr,gt)

-2.47086591009682

In [304]:
metrics.compute_rce(pr,gt)

-0.07617240971122818

### Rough Testing Below

In [ ]:
for lst in ast.literal_eval(user_tokens_val_set.loc[46166]['text_ tokens_y']):
    list_of_tokens = list(map(int, lst.split('\t')))
    if len(list_of_tokens) > 512:
        pre_list = list_of_tokens[:511]
        post_list = list_of_tokens[511-len(list_of_tokens):]
        post_list.insert(0, 101)
        pre_token_list_embeddings = model(torch.tensor(pre_list).unsqueeze(0))[0][0][0]
        post_token_list_embeddings = model(torch.tensor(post_list).unsqueeze(0))[0][0][0]
        token_list_embeddings = pre_token_list_embeddings + post_token_list_embeddings)/2.0
    #token_list_embeddings = model(torch.tensor(list_of_tokens).unsqueeze(0))[0][0][0]

In [ ]:
df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

In [ ]:
user_tokens.to_csv('s3://recsyschallenge2020/user_tokens_embeddins.csv', index = False)

In [ ]:
user_tokens.head()

In [ ]:
len(user_tokens)

In [ ]:
input_ids = torch.tensor([101, 108, 31278, 90939, 70325, 196, 199, 71436, 10107, 29190, 10107, 106, 16680, 68314, 10153, 17015, 15934, 10104, 108, 10233, 12396, 14945, 10107, 10858, 11405, 13600, 13597, 169, 57343, 64482, 119, 119, 119, 100, 11741, 16381, 10109, 68830, 10110, 20886, 108, 10233, 11127, 21768, 100, 14120, 131, 120, 120, 188, 119, 11170, 120, 12132, 10884, 10157, 11490, 12022, 10113, 10731, 10729, 11565, 14120, 131, 120, 120, 188, 119, 11170, 120, 162, 11211, 11703, 12022, 11211, 10240, 44466, 100886, 102]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0][0][0]  # The last hidden-state is the first element of the output tuple

In [ ]:
model(torch.tensor(list(map(int, ast.literal_eval(user_tokens.loc[0]['text_ tokens']).split('\t')))).unsqueeze(0))[0][0][0]

In [ ]:
it = 0

start = time.time()

# your code

# end
sum_tensors = torch.zeros([768], dtype=torch.float32)
for index, row in user_tokens.iterrows():
    #print(len(row))
    tweet_token_list = ast.literal_eval(row['text_ tokens'])
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    if it %100 == 0:
        print(f'Time: {time.time() - start}')
        print(it)
    it = it + 1
    print(it)
    avg = sum_tensors/len(row)

In [ ]:
sum_tensors = torch.zeros([768], dtype=torch.float32)

In [ ]:
sum_tensors

In [ ]:
import dask.dataframe as ddf

In [ ]:
df_dask = ddf.from_pandas(user_tokens, npartitions=32)   # where the number of partitions is the number of cores you want to use

In [ ]:
df_dask.apply(lambda x: calculate_average(x), axis  = 1).compute(scheduler='multiprocessing')